# 🐘 Introduction au Big Data & Systèmes Distribués

---

## 📋 Prérequis

| Niveau | Compétence |
|--------|------------|
| ✅ Requis | Avoir suivi le module `06_intro_relational_databases` |
| ✅ Requis | Avoir suivi le module `07_sql_for_data_engineers` |

## 🎯 Objectifs du module

À la fin de ce notebook, tu seras capable de :

- ✅ Comprendre ce qu'est le Big Data et ses caractéristiques (5V)
- ✅ Expliquer pourquoi le traitement distribué est nécessaire
- ✅ Décrire l'architecture Hadoop (HDFS, MapReduce, YARN)
- ✅ Comprendre le modèle MapReduce
- ✅ Expliquer pourquoi Spark a remplacé MapReduce
- ✅ **Comprendre le NoSQL et ses différents types**
- ✅ **Savoir quand utiliser SQL vs NoSQL**
- ✅ **Connaître le théorème CAP**

---

> 💡 **Note** : Ce module est théorique. La pratique viendra avec MongoDB (module suivant) et PySpark !

---

## 🌍 1. C'est quoi le Big Data ?

### 📖 Définition

Le **Big Data** désigne des ensembles de données **tellement volumineux et complexes** qu'ils ne peuvent pas être traités par des outils traditionnels (Excel, bases SQL classiques, un seul serveur).

### 📅 Origine

Le terme a émergé dans les années **2000** avec l'explosion :

- 🌐 D'Internet et des réseaux sociaux
- 📱 Des smartphones
- 📡 Des capteurs IoT
- 💳 Des transactions en ligne

### 📊 Ordres de grandeur

```
1 Ko  (Kilooctet)   = 1 page de texte
1 Mo  (Mégaoctet)   = 1 photo HD
1 Go  (Gigaoctet)   = 1 film HD
1 To  (Téraoctet)   = 1 000 films HD
1 Po  (Pétaoctet)   = 1 000 To = 1 million de Go
1 Eo  (Exaoctet)    = 1 000 Po
1 Zo  (Zettaoctet)  = 1 000 Eo
```

### 🏢 Exemples concrets

| Entreprise | Volume de données |
|------------|-------------------|
| Facebook | ~4 **Po** générés par jour |
| Google | ~20 **Po** traités par jour |
| Netflix | ~60 **Po** de vidéos stockées |
| CERN (LHC) | ~1 **Po** par seconde pendant les expériences |

---

## 📊 2. Les 5V du Big Data

Les caractéristiques du Big Data sont souvent résumées par les **5V** :

```
                    ┌─────────────┐
                    │   VOLUME    │
                    │  (quantité) │
                    └──────┬──────┘
                           │
        ┌──────────────────┼──────────────────┐
        │                  │                  │
        ▼                  ▼                  ▼
┌───────────────┐  ┌───────────────┐  ┌───────────────┐
│   VELOCITY    │  │   VARIETY     │  │   VERACITY    │
│   (vitesse)   │  │  (diversité)  │  │  (fiabilité)  │
└───────────────┘  └───────────────┘  └───────────────┘
                           │
                           ▼
                    ┌─────────────┐
                    │    VALUE    │
                    │  (valeur)   │
                    └─────────────┘
```

---

### 📦 Volume — La quantité massive de données

| Défi | Solution |
|------|----------|
| Impossible de stocker sur un seul disque | Stockage distribué (HDFS, S3) |
| Impossible de charger en RAM | Traitement par partitions |

---

### ⚡ Velocity — La vitesse de génération et traitement

| Type | Exemple | Latence |
|------|---------|--------|
| **Batch** | Rapport mensuel | Heures |
| **Near real-time** | Dashboard | Minutes |
| **Real-time / Streaming** | Détection de fraude | Millisecondes |

---

### 🎨 Variety — La diversité des formats

> 💡 **Rappel** : Tu as déjà vu ça dans le module 06 !

| Type | Format | Exemple |
|------|--------|--------|
| **Structuré** | Tables, colonnes fixes | SQL, CSV |
| **Semi-structuré** | Schéma flexible | JSON, XML (MongoDB, Elasticsearch) |
| **Non-structuré** | Pas de schéma | Images, vidéos, texte libre |

---

### ✅ Veracity — La fiabilité des données

| Problème | Impact |
|----------|--------|
| Données manquantes | Résultats biaisés |
| Doublons | Comptages faux |
| Erreurs de saisie | Mauvaises décisions |
| Données obsolètes | Analyses non pertinentes |

> 💡 C'est là que le **Data Engineer** intervient : nettoyer, valider, transformer !

---

### 💎 Value — La valeur extraite

Les données n'ont de valeur que si on peut en tirer des **insights** :
- 📈 Prédictions (ML)
- 📊 Dashboards
- 🔔 Alertes
- 💰 Optimisation business

---

## ⚙️ 3. Pourquoi le traitement distribué ?

### 🚫 Limites d'une machine unique

Imaginons que tu dois traiter **10 To** de logs :

| Ressource | Limite typique | Problème |
|-----------|---------------|----------|
| **RAM** | 64-256 Go | 10 To ne tient pas en mémoire |
| **CPU** | 8-64 cœurs | Traitement séquentiel = trop lent |
| **Disque** | 500 Mo/s lecture | 10 To = 5+ heures juste pour lire |
| **Réseau** | Goulot d'étranglement | Transférer 10 To = des heures |

---

### 📈 Scale-Up vs Scale-Out

```
SCALE-UP (vertical)              SCALE-OUT (horizontal)
─────────────────────            ─────────────────────────

   ┌─────────────┐               ┌───┐ ┌───┐ ┌───┐ ┌───┐
   │             │               │   │ │   │ │   │ │   │
   │   MEGA      │               │ S │ │ S │ │ S │ │ S │
   │  SERVEUR    │      vs       │ 1 │ │ 2 │ │ 3 │ │ 4 │
   │   💪💪💪    │               │   │ │   │ │   │ │   │
   │             │               └───┘ └───┘ └───┘ └───┘
   └─────────────┘               
                                 Cluster de serveurs
   + Plus de RAM                 + Moins cher (commodity)
   + Plus de CPU                 + Scalabilité infinie
   - Très cher $$$               + Tolérance aux pannes
   - Limite physique             - Plus complexe
```

### ✅ Le Big Data utilise le Scale-Out !

Au lieu d'une machine surpuissante, on utilise un **cluster** de machines ordinaires.

---

## 🧠 4. Concepts clés du traitement distribué

### 🔄 Parallélisme vs Distribution

| Concept | Description | Exemple |
|---------|-------------|--------|
| **Parallélisme** | Plusieurs tâches en même temps sur **une machine** | Multi-threading |
| **Distribution** | Tâches réparties sur **plusieurs machines** | Cluster Hadoop/Spark |

---

### 📍 Data Locality — "Amener le code aux données"

```
❌ MAUVAIS : Déplacer les données vers le code
┌────────────────┐                ┌────────────────┐
│   Serveur 1    │  ══10 To══►    │   Serveur 2    │
│   (données)    │   réseau       │    (calcul)    │
└────────────────┘   lent !       └────────────────┘

✅ BON : Déplacer le code vers les données
┌────────────────┐                ┌────────────────┐
│   Serveur 1    │  ◄══code══     │   Serveur 2    │
│ données+calcul │   (petit)      │    (master)    │
└────────────────┘                └────────────────┘
```

> 💡 C'est le principe fondamental de Hadoop et Spark !

---

### 🛡️ Fault Tolerance — Tolérance aux pannes

Dans un cluster de 1000 machines, **des pannes arrivent tous les jours** !

| Stratégie | Description |
|-----------|-------------|
| **Réplication** | Copier les données sur plusieurs nœuds (HDFS : 3 copies) |
| **Checkpointing** | Sauvegarder l'état intermédiaire |
| **Lineage** | Recalculer les données perdues (Spark RDD) |

---

## 🐘 5. L'écosystème Hadoop

**Hadoop** est un framework open-source créé par Yahoo (2006), inspiré des papiers de Google (GFS, MapReduce).

### 🏗️ Architecture

```
┌─────────────────────────────────────────────────────────────┐
│                    ÉCOSYSTÈME HADOOP                        │
├─────────────────────────────────────────────────────────────┤
│                                                             │
│   ┌─────────┐  ┌─────────┐  ┌─────────┐  ┌─────────┐       │
│   │  Hive   │  │   Pig   │  │  HBase  │  │  Sqoop  │  ...  │
│   │  (SQL)  │  │(scripts)│  │ (NoSQL) │  │ (import)│       │
│   └────┬────┘  └────┬────┘  └────┬────┘  └────┬────┘       │
│        │            │            │            │             │
│        └────────────┴─────┬──────┴────────────┘             │
│                           │                                 │
│                    ┌──────┴──────┐                         │
│                    │  MapReduce  │  ◄── Traitement         │
│                    │  (calcul)   │                         │
│                    └──────┬──────┘                         │
│                           │                                 │
│                    ┌──────┴──────┐                         │
│                    │    YARN     │  ◄── Ressources         │
│                    │ (scheduler) │                         │
│                    └──────┬──────┘                         │
│                           │                                 │
│                    ┌──────┴──────┐                         │
│                    │    HDFS     │  ◄── Stockage           │
│                    │  (fichiers) │                         │
│                    └─────────────┘                         │
│                                                             │
└─────────────────────────────────────────────────────────────┘
```

---

### 📁 HDFS — Hadoop Distributed File System

Système de fichiers distribué qui stocke les données sur plusieurs machines.

```
Fichier original : data.csv (300 Mo)
                      │
        ┌─────────────┼─────────────┐
        ▼             ▼             ▼
   ┌─────────┐   ┌─────────┐   ┌─────────┐
   │ Block 1 │   │ Block 2 │   │ Block 3 │   (128 Mo chacun)
   │ 128 Mo  │   │ 128 Mo  │   │  44 Mo  │
   └────┬────┘   └────┬────┘   └────┬────┘
        │             │             │
   Répliqué 3x   Répliqué 3x   Répliqué 3x
```

| Caractéristique | Valeur par défaut |
|-----------------|-------------------|
| Taille de bloc | 128 Mo |
| Facteur de réplication | 3 |
| Type d'accès | Write once, read many |

---

### 🎛️ YARN — Yet Another Resource Negotiator

Gestionnaire de ressources du cluster :

- Alloue CPU/RAM aux applications
- Gère la file d'attente des jobs
- Surveille l'état des nœuds

---

## 🔄 6. MapReduce — Le modèle de traitement

**MapReduce** est un modèle de programmation pour traiter de grandes quantités de données en parallèle.

### 📐 Les 3 étapes

| Étape | Action | Parallélisme |
|-------|--------|-------------|
| **Map** | Transformer chaque élément | ✅ Parallèle |
| **Shuffle** | Regrouper par clé | ⚠️ Réseau |
| **Reduce** | Agréger les valeurs | ✅ Parallèle |

---

### 📝 Exemple : Word Count

Compter les occurrences de chaque mot dans un texte.

```
ENTRÉE : "hello world hello"

═══════════════════════════════════════════════════════════════════
                              MAP
═══════════════════════════════════════════════════════════════════

  "hello world hello"   
         │
         ▼
  ┌──────────────────────────────────────┐
  │  (hello, 1)  (world, 1)  (hello, 1)  │
  └──────────────────────────────────────┘

═══════════════════════════════════════════════════════════════════
                            SHUFFLE
═══════════════════════════════════════════════════════════════════

  Regrouper par clé (mot) :
  
  hello → [1, 1]
  world → [1]

═══════════════════════════════════════════════════════════════════
                            REDUCE
═══════════════════════════════════════════════════════════════════

  hello → sum([1, 1]) → 2
  world → sum([1])    → 1

═══════════════════════════════════════════════════════════════════

SORTIE : { "hello": 2, "world": 1 }
```

---

### 🖥️ MapReduce sur un cluster

```
                         ┌─────────────┐
                         │   MASTER    │
                         │  (Driver)   │
                         └──────┬──────┘
                                │
           ┌────────────────────┼────────────────────┐
           │                    │                    │
           ▼                    ▼                    ▼
    ┌─────────────┐      ┌─────────────┐      ┌─────────────┐
    │   NODE 1    │      │   NODE 2    │      │   NODE 3    │
    ├─────────────┤      ├─────────────┤      ├─────────────┤
    │ Données:    │      │ Données:    │      │ Données:    │
    │ "hello"     │      │ "world"     │      │ "hello"     │
    ├─────────────┤      ├─────────────┤      ├─────────────┤
    │ MAP:        │      │ MAP:        │      │ MAP:        │
    │ (hello, 1)  │      │ (world, 1)  │      │ (hello, 1)  │
    └──────┬──────┘      └──────┬──────┘      └──────┬──────┘
           │                    │                    │
           └────────────────────┼────────────────────┘
                                │
                         SHUFFLE (réseau)
                                │
           ┌────────────────────┴────────────────────┐
           ▼                                         ▼
    ┌─────────────┐                           ┌─────────────┐
    │  REDUCER 1  │                           │  REDUCER 2  │
    ├─────────────┤                           ├─────────────┤
    │ hello: [1,1]│                           │ world: [1]  │
    │ → hello: 2  │                           │ → world: 1  │
    └─────────────┘                           └─────────────┘
```

> 💡 Chaque nœud traite **ses données localement** (data locality) !

---

## ⚠️ 7. Limites de MapReduce

### 😓 Problèmes de MapReduce

| Problème | Cause | Impact |
|----------|-------|--------|
| **Lent** | Écrit sur disque entre chaque étape | I/O intensif |
| **Verbose** | Code Java complexe | Productivité basse |
| **Batch only** | Pas de streaming | Pas de temps réel |
| **Pas de cache** | Relit les données à chaque job | Itérations lentes (ML) |

### 💾 Le problème du disque

```
MapReduce : DISQUE → Map → DISQUE → Shuffle → DISQUE → Reduce → DISQUE
                 ↑           ↑              ↑              ↑
                 └───────────┴──────────────┴──────────────┘
                              LENT ! (I/O disque)

Spark :     DISQUE → Map → MÉMOIRE → Shuffle → MÉMOIRE → Reduce
                            ↑                    ↑
                            └────────────────────┘
                              RAPIDE ! (in-memory)
```

---

### 🔥 Exemple : Algorithme itératif (ML)

Pour un algorithme qui fait 10 itérations sur les mêmes données :

| Framework | Comportement | Temps |
|-----------|-------------|-------|
| **MapReduce** | Relit les données du disque 10 fois | 😓 |
| **Spark** | Garde les données en mémoire, itère 10 fois | ⚡ |

---

## ⚡ 8. Spark — L'évolution

**Apache Spark** (2014) a été créé pour résoudre les limitations de MapReduce.

### 📊 Comparaison Hadoop MapReduce vs Spark

| Critère | Hadoop MapReduce | Apache Spark |
|---------|------------------|---------------|
| **Vitesse** | Lent (disque) | 100x plus rapide (mémoire) |
| **Facilité** | Java verbeux | Python, Scala, SQL |
| **Traitement** | Batch only | Batch + Streaming |
| **Itérations** | Lent (relit le disque) | Rapide (cache en RAM) |
| **Écosystème** | Hive, Pig, etc. | Spark SQL, MLlib, GraphX |
| **Stockage** | HDFS | HDFS, S3, Cassandra, etc. |

---

### 🏗️ Architecture Spark

```
┌─────────────────────────────────────────────────────────────┐
│                      APACHE SPARK                           │
├─────────────────────────────────────────────────────────────┤
│                                                             │
│   ┌───────────┐ ┌───────────┐ ┌───────────┐ ┌───────────┐  │
│   │ Spark SQL │ │ Streaming │ │   MLlib   │ │  GraphX   │  │
│   │  (SQL)    │ │ (temps    │ │   (ML)    │ │ (graphes) │  │
│   │           │ │   réel)   │ │           │ │           │  │
│   └─────┬─────┘ └─────┬─────┘ └─────┬─────┘ └─────┬─────┘  │
│         │             │             │             │        │
│         └─────────────┴──────┬──────┴─────────────┘        │
│                              │                              │
│                       ┌──────┴──────┐                      │
│                       │ Spark Core  │                      │
│                       │   (RDD)     │                      │
│                       └──────┬──────┘                      │
│                              │                              │
│         ┌────────────────────┼────────────────────┐        │
│         ▼                    ▼                    ▼        │
│   ┌──────────┐        ┌──────────┐        ┌──────────┐    │
│   │   YARN   │        │   Mesos  │        │Standalone│    │
│   └──────────┘        └──────────┘        └──────────┘    │
│                                                             │
└─────────────────────────────────────────────────────────────┘
```

> 💡 Spark peut fonctionner **sur YARN** (cluster Hadoop existant) ou en mode standalone.

---

### 🔮 Concepts Spark à venir 

| Concept | Description |
|---------|-------------|
| **RDD** | Resilient Distributed Dataset — collection distribuée |
| **DataFrame** | Comme un tableau avec colonnes (similaire à Pandas) |
| **Transformation** | Opération lazy (map, filter, groupBy) |
| **Action** | Déclenche le calcul (collect, count, show) |
| **Lazy Evaluation** | Rien ne s'exécute tant qu'une action n'est pas appelée |
| **Partition** | Morceau de données traité par un worker |

---

### 🎯 Ce que tu vas apprendre avec PySpark

- Créer et manipuler des DataFrames distribués
- Écrire des transformations SQL-like
- Lire/écrire des fichiers (CSV, Parquet, JSON)
- Optimiser les performances
- Construire des pipelines de données

---

# 🗄️ NoSQL — Au-delà des bases relationnelles

Jusqu'ici, tu as travaillé avec des bases **relationnelles** (SQL). Mais le Big Data a fait émerger un nouveau paradigme : **NoSQL**.

---

## 🤔 Pourquoi NoSQL ?

### Les limites des bases relationnelles face au Big Data

```
┌─────────────────────────────────────────────────────────────────┐
│              PROBLÈMES DES BASES SQL CLASSIQUES                 │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  ❌ Schéma rigide → Difficile de changer la structure           │
│  ❌ Scale-up only → Un seul serveur (coûteux)                   │
│  ❌ Jointures → Lentes sur des milliards de lignes              │
│  ❌ ACID strict → Latence élevée                                │
│  ❌ Données variées → JSON, graphes mal gérés                   │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

### La solution NoSQL

```
┌─────────────────────────────────────────────────────────────────┐
│                    AVANTAGES NoSQL                              │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  ✅ Schéma flexible → S'adapte aux données                      │
│  ✅ Scale-out natif → Ajout de serveurs facile                  │
│  ✅ Pas de jointures → Données dénormalisées, rapides           │
│  ✅ Haute disponibilité → Tolérance aux pannes                  │
│  ✅ Modèles variés → Document, clé-valeur, graphe...            │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

> 💡 **NoSQL** = "Not Only SQL" (pas seulement SQL), pas "No SQL" !

---

## 📊 Les 4 types de bases NoSQL

```
┌─────────────────────────────────────────────────────────────────────────────┐
│                         TYPES DE BASES NoSQL                                │
├─────────────────────┬─────────────────────┬─────────────────────┬───────────┤
│     📄 DOCUMENT     │   🔑 CLÉ-VALEUR    │    📊 COLONNES      │  🔗 GRAPHE │
├─────────────────────┼─────────────────────┼─────────────────────┼───────────┤
│                     │                     │                     │           │
│  {                  │   key1 → value1     │  Row1: col1, col2   │   (A)──→(B)│
│    "nom": "Alice", │   key2 → value2     │  Row2: col1, col3   │    │       │
│    "age": 30        │   key3 → value3     │  Row3: col2, col4   │    ↓       │
│  }                  │                     │                     │   (C)     │
│                     │                     │                     │           │
├─────────────────────┼─────────────────────┼─────────────────────┼───────────┤
│  MongoDB            │  Redis              │  Cassandra          │ Neo4j     │
│  Couchbase          │  Memcached          │  HBase              │ Amazon    │
│  Firestore          │  DynamoDB           │  ScyllaDB           │ Neptune   │
└─────────────────────┴─────────────────────┴─────────────────────┴───────────┘
```

---

### 📄 1. Bases Document (MongoDB, Couchbase)

Stockent des **documents JSON/BSON** avec un schéma flexible.

```json
// Document MongoDB
{
  "_id": "user123",
  "nom": "Alice Dupont",
  "email": "alice@example.com",
  "adresses": [
    {"type": "domicile", "ville": "Paris"},
    {"type": "travail", "ville": "Lyon"}
  ],
  "commandes": [
    {"id": "cmd001", "total": 150.00},
    {"id": "cmd002", "total": 89.99}
  ]
}
```

| Avantages | Inconvénients |
|-----------|---------------|
| Schéma flexible | Pas de jointures natives |
| Documents imbriqués | Données dupliquées |
| Requêtes riches | Transactions limitées |

**Cas d'usage** : Catalogues produits, profils utilisateurs, CMS, logs

---

### 🔑 2. Bases Clé-Valeur (Redis, Memcached, DynamoDB)

Stockage ultra-simple : une **clé** → une **valeur**.

```
┌─────────────────┬────────────────────────────┐
│      CLÉ        │          VALEUR            │
├─────────────────┼────────────────────────────┤
│  user:123       │  {"nom": "Alice", ...}    │
│  session:abc    │  {"user_id": 123, ...}    │
│  cache:page:42  │  "<html>...</html>"       │
│  counter:views  │  1583920                   │
└─────────────────┴────────────────────────────┘
```

| Avantages | Inconvénients |
|-----------|---------------|
| Ultra-rapide (RAM) | Requêtes limitées |
| Simple | Pas de structure |
| Scalable | Recherche par clé uniquement |

**Cas d'usage** : Cache, sessions, compteurs, files d'attente

---

### 📊 3. Bases Colonnes (Cassandra, HBase, ScyllaDB)

Optimisées pour les **écritures massives** et les **requêtes analytiques**.

```
┌──────────────────────────────────────────────────────┐
│  Row Key: user123                                    │
├──────────────┬──────────────┬──────────────┬─────────┤
│  nom:Alice   │  age:30      │  ville:Paris │  ...    │
├──────────────────────────────────────────────────────┤
│  Row Key: user456                                    │
├──────────────┬──────────────┬──────────────────────────┤
│  nom:Bob     │  email:...   │  (pas de ville)          │
└──────────────┴──────────────┴──────────────────────────┘
```

| Avantages | Inconvénients |
|-----------|---------------|
| Écritures très rapides | Modèle complexe |
| Scalabilité linéaire | Requêtes limitées |
| Tolérance aux pannes | Pas de jointures |

**Cas d'usage** : IoT, time-series, logs, analytics

---

### 🔗 4. Bases Graphe (Neo4j, Amazon Neptune, ArangoDB)

Optimisées pour les **relations complexes** entre entités.

```
         ┌─────────┐
         │  Alice  │
         └────┬────┘
              │ KNOWS
              ▼
         ┌─────────┐       WORKS_AT       ┌─────────────┐
         │   Bob   │ ──────────────────►  │  TechCorp   │
         └────┬────┘                      └─────────────┘
              │ FOLLOWS
              ▼
         ┌─────────┐
         │ Charlie │
         └─────────┘
```

| Avantages | Inconvénients |
|-----------|---------------|
| Relations naturelles | Scalabilité complexe |
| Requêtes de traversée rapides | Cas d'usage spécifique |
| Flexibilité | Moins mature |

**Cas d'usage** : Réseaux sociaux, recommandations, fraude, connaissances

---

## 🔺 Le théorème CAP

Le **théorème CAP** (Eric Brewer, 2000) est fondamental pour comprendre les choix de design des bases distribuées.

```
                         C
                    Consistency
                    (Cohérence)
                        /\
                       /  \
                      /    \
                     /  CA  \
                    /________\
                   /\        /\
                  /  \      /  \
                 / CP \    / AP \
                /______\  /______\
               A                    P
          Availability          Partition
         (Disponibilité)        Tolerance
                              (Tolérance aux
                                partitions)
```

### 📖 Les 3 propriétés

| Propriété | Signification | Exemple |
|-----------|---------------|--------|
| **C**onsistency | Tous les nœuds voient les mêmes données | Lecture après écriture identique |
| **A**vailability | Le système répond toujours | Pas de timeout |
| **P**artition Tolerance | Fonctionne malgré des pannes réseau | Serveurs isolés continuent |

### ⚠️ Le théorème dit :

> **En cas de partition réseau, tu dois choisir entre Cohérence et Disponibilité.**

Tu ne peux avoir que **2 sur 3** !

---

### 🗂️ Classification des bases selon CAP

| Type | Choix CAP | Bases | Comportement |
|------|-----------|-------|-------------|
| **CP** | Cohérence + Partition | MongoDB, HBase, Redis | Peut refuser des requêtes si partition |
| **AP** | Disponibilité + Partition | Cassandra, DynamoDB, CouchDB | Répond toujours, cohérence éventuelle |
| **CA** | Cohérence + Disponibilité | PostgreSQL, MySQL (single node) | Pas de tolérance aux partitions |

---

### 💡 En pratique : Cohérence éventuelle (Eventual Consistency)

La plupart des bases NoSQL utilisent la **cohérence éventuelle** :

```
Temps T0: Écriture sur Nœud A
          [A: nouvelle valeur] [B: ancienne] [C: ancienne]
          
Temps T1: Réplication en cours
          [A: nouvelle valeur] [B: nouvelle] [C: ancienne]
          
Temps T2: Cohérence atteinte
          [A: nouvelle valeur] [B: nouvelle] [C: nouvelle]
```

> 💡 Acceptable pour : likes, vues, logs. Pas pour : transactions bancaires !

---

## ⚖️ SQL vs NoSQL — Quand utiliser quoi ?

### 📊 Tableau comparatif

| Critère | SQL (Relationnel) | NoSQL |
|---------|-------------------|-------|
| **Schéma** | Fixe, défini à l'avance | Flexible, dynamique |
| **Données** | Structurées, normalisées | Semi/non-structurées |
| **Relations** | Jointures natives | Données embarquées/dénormalisées |
| **Transactions** | ACID complet | BASE (éventuel) |
| **Scalabilité** | Verticale (scale-up) | Horizontale (scale-out) |
| **Requêtes** | SQL standardisé | Langage spécifique |
| **Maturité** | 40+ ans | 15+ ans |

---

### ✅ Utilise SQL quand...

| Situation | Exemple |
|-----------|--------|
| Données très structurées | Comptabilité, RH |
| Relations complexes | ERP, CRM |
| Transactions critiques | Banque, e-commerce |
| Requêtes ad-hoc variées | Reporting, BI |
| Intégrité des données critique | Santé, finance |

---

### ✅ Utilise NoSQL quand...

| Situation | Type NoSQL | Exemple |
|-----------|------------|--------|
| Schéma variable | Document | Catalogues, profils |
| Cache haute performance | Clé-valeur | Sessions, cache |
| Écritures massives | Colonnes | IoT, logs |
| Relations complexes | Graphe | Réseaux sociaux |
| Scale-out nécessaire | Tous | Big Data |
| Développement agile | Document | Startups, MVPs |

---

### 🎯 En réalité : on utilise les deux !

```
┌─────────────────────────────────────────────────────────────────┐
│                  ARCHITECTURE POLYGLOTTE                        │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│   [Application Web]                                             │
│         │                                                       │
│         ├──► PostgreSQL (utilisateurs, commandes, paiements)   │
│         │                                                       │
│         ├──► MongoDB (catalogues produits, reviews)            │
│         │                                                       │
│         ├──► Redis (sessions, cache, rate limiting)            │
│         │                                                       │
│         ├──► Elasticsearch (recherche full-text)               │
│         │                                                       │
│         └──► Neo4j (recommandations, graphe social)            │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

> 💡 **Polyglot Persistence** : Utiliser la bonne base pour le bon cas d'usage !

---

## 🗂️ Bases NoSQL populaires

### 📄 MongoDB — La plus populaire (Document)

| Caractéristique | Description |
|-----------------|-------------|
| **Type** | Base de documents JSON (BSON) |
| **Langage de requête** | MQL (MongoDB Query Language) |
| **Scalabilité** | Sharding natif |
| **Transactions** | ACID multi-documents (depuis v4.0) |
| **Cloud** | MongoDB Atlas |

```javascript
// Exemple de requête MongoDB
db.users.find({
  age: { $gte: 25 },
  ville: "Paris"
}).sort({ nom: 1 })
```

> 💡 **Tu vas apprendre MongoDB en détail dans le module suivant !**

---

### ⚡ Redis — Ultra-rapide (Clé-valeur)

| Caractéristique | Description |
|-----------------|-------------|
| **Type** | In-memory key-value store |
| **Performance** | < 1ms de latence |
| **Structures** | Strings, Lists, Sets, Hashes, Streams |
| **Persistance** | Optionnelle (RDB, AOF) |

```python
# Exemple avec Python
import redis
r = redis.Redis()

r.set("user:123:name", "Alice")
r.get("user:123:name")  # b'Alice'

r.incr("page:views")  # Compteur atomique
```

---

### 🌐 Cassandra — Écritures massives (Colonnes)

| Caractéristique | Description |
|-----------------|-------------|
| **Type** | Wide-column store |
| **Origine** | Facebook (inbox search) |
| **Scalabilité** | Linéaire, peer-to-peer |
| **Disponibilité** | Pas de single point of failure |

```sql
-- CQL (Cassandra Query Language)
CREATE TABLE events (
    device_id UUID,
    timestamp TIMESTAMP,
    value DOUBLE,
    PRIMARY KEY (device_id, timestamp)
);
```

---

### 🔍 Elasticsearch — Recherche (Document + Search)

| Caractéristique | Description |
|-----------------|-------------|
| **Type** | Moteur de recherche distribué |
| **Base** | Apache Lucene |
| **Usage** | Full-text search, logs, analytics |
| **Stack** | ELK (Elasticsearch, Logstash, Kibana) |

> 💡 **Tu as vu Elasticsearch dans le module 10 !**

---

## 🧠 Quiz NoSQL

### ❓ Q8. Que signifie NoSQL ?
a) No SQL (pas de SQL)  
b) Not Only SQL  
c) New SQL  
d) Non-Structured Query Language

<details><summary>💡 Réponse</summary>

✅ **b** — NoSQL = "Not Only SQL" (pas seulement SQL). Ces bases peuvent parfois supporter du SQL !
</details>

---

### ❓ Q9. Quel type de base NoSQL est MongoDB ?
a) Clé-valeur  
b) Colonnes  
c) Document  
d) Graphe

<details><summary>💡 Réponse</summary>

✅ **c** — MongoDB est une base de **documents** (JSON/BSON).
</details>

---

### ❓ Q10. Selon le théorème CAP, que sacrifie une base AP ?
a) Disponibilité  
b) Cohérence  
c) Tolérance aux partitions  
d) Performance

<details><summary>💡 Réponse</summary>

✅ **b** — Une base AP (Available + Partition tolerant) sacrifie la Cohérence immédiate. Elle utilise la "cohérence éventuelle".
</details>

---

### ❓ Q11. Pour quel cas d'usage Redis est-il le plus adapté ?
a) Stockage de documents JSON  
b) Cache et sessions  
c) Analyse de graphes sociaux  
d) Stockage de vidéos

<details><summary>💡 Réponse</summary>

✅ **b** — Redis est une base in-memory clé-valeur, idéale pour le cache et les sessions.
</details>

---

### ❓ Q12. Quelle base NoSQL choisir pour des relations complexes (réseau social) ?
a) MongoDB  
b) Redis  
c) Cassandra  
d) Neo4j

<details><summary>💡 Réponse</summary>

✅ **d** — Neo4j est une base de **graphes**, optimisée pour les relations complexes.
</details>

---

## 📋 Résumé

### Les 5V du Big Data

| V | Défi |
|---|------|
| **Volume** | Stocker et traiter des To/Po |
| **Velocity** | Traiter en temps réel |
| **Variety** | Gérer tous les formats |
| **Veracity** | Assurer la qualité |
| **Value** | Extraire des insights |

### Traitement distribué

| Concept | Retenir |
|---------|--------|
| **Scale-out** | Cluster de machines ordinaires |
| **Data locality** | Amener le code aux données |
| **Fault tolerance** | Réplication, recalcul |

### Hadoop vs Spark

| | Hadoop MR | Spark |
|---|-----------|-------|
| Stockage intermédiaire | Disque | Mémoire |
| Vitesse | Lent | 100x plus rapide |
| Langages | Java | Python, Scala, SQL |

### SQL vs NoSQL

| Type | Quand utiliser |
|------|----------------|
| **SQL** | Données structurées, transactions, intégrité |
| **NoSQL Document** | Schéma flexible, JSON (MongoDB) |
| **NoSQL Clé-valeur** | Cache, sessions (Redis) |
| **NoSQL Colonnes** | Écritures massives, IoT (Cassandra) |
| **NoSQL Graphe** | Relations complexes (Neo4j) |

### Théorème CAP

| Choix | Exemples |
|-------|----------|
| **CP** | MongoDB, HBase |
| **AP** | Cassandra, DynamoDB |
| **CA** | PostgreSQL (single node) |

---

## 🧠 Quiz

---

### ❓ Q1. Quels sont les 3V originaux du Big Data ?
a) Vitesse, Valeur, Vérité  
b) Volume, Velocity, Variety  
c) Volume, Validation, Visualisation  
d) Vélocité, Véracité, Valorisation

<details><summary>💡 Réponse</summary>

✅ **b** — Les 3V originaux (Doug Laney, 2001) sont Volume, Velocity, Variety.
</details>

---

### ❓ Q2. Que signifie "Scale-out" ?
a) Augmenter la RAM d'un serveur  
b) Ajouter des machines au cluster  
c) Compresser les données  
d) Réduire la taille du cluster

<details><summary>💡 Réponse</summary>

✅ **b** — Scale-out (horizontal) = ajouter des machines. Scale-up (vertical) = augmenter les ressources d'une machine.
</details>

---

### ❓ Q3. Quel est le principe de "Data Locality" ?
a) Stocker les données localement sur son PC  
b) Déplacer les données vers le serveur de calcul  
c) Déplacer le code vers les données  
d) Compresser les données pour les transférer

<details><summary>💡 Réponse</summary>

✅ **c** — On envoie le code (petit) vers les données (grosses), pas l'inverse.
</details>

---

### ❓ Q4. Quels sont les 3 composants principaux de Hadoop ?
a) HDFS, Spark, Kafka  
b) HDFS, MapReduce, YARN  
c) Hive, Pig, HBase  
d) Map, Shuffle, Reduce

<details><summary>💡 Réponse</summary>

✅ **b** — HDFS (stockage), MapReduce (calcul), YARN (ressources).
</details>

---

### ❓ Q5. Pourquoi Spark est plus rapide que MapReduce ?
a) Il utilise un meilleur algorithme  
b) Il stocke les données intermédiaires en mémoire  
c) Il compresse les données  
d) Il utilise plus de serveurs

<details><summary>💡 Réponse</summary>

✅ **b** — Spark garde les données en mémoire (RAM) au lieu d'écrire sur disque entre chaque étape.
</details>

---

### ❓ Q6. Dans MapReduce, que fait l'étape "Shuffle" ?
a) Trier les données  
b) Regrouper les données par clé  
c) Supprimer les doublons  
d) Compresser les résultats

<details><summary>💡 Réponse</summary>

✅ **b** — Le Shuffle regroupe toutes les valeurs ayant la même clé pour les envoyer au même Reducer.
</details>

---

### ❓ Q7. Combien de copies HDFS fait-il par défaut ?
a) 1  
b) 2  
c) 3  
d) 5

<details><summary>💡 Réponse</summary>

✅ **c** — HDFS réplique chaque bloc 3 fois par défaut pour la tolérance aux pannes.
</details>

---

## 📚 Ressources

### Big Data & Hadoop
- [Hadoop Documentation](https://hadoop.apache.org/docs/stable/)
- [Spark Documentation](https://spark.apache.org/docs/latest/)
- [The Google File System (paper)](https://research.google/pubs/pub51/)
- [MapReduce: Simplified Data Processing (paper)](https://research.google/pubs/pub62/)

### NoSQL
- [MongoDB University](https://university.mongodb.com/) — Cours gratuits
- [Redis Documentation](https://redis.io/docs/)
- [Cassandra Documentation](https://cassandra.apache.org/doc/latest/)
- [CAP Theorem Explained](https://www.ibm.com/topics/cap-theorem)

---

## ➡️ Prochaine étape

Tu as maintenant les bases théoriques du Big Data, du traitement distribué et du NoSQL. Place à la pratique avec MongoDB !

👉 **Module suivant : `09_mongodb_for_data_engineers.ipynb`** — MongoDB pour les Data Engineers

Tu apprendras à :
- Installer et utiliser MongoDB
- Créer des collections et documents
- Écrire des requêtes MQL (MongoDB Query Language)
- Utiliser PyMongo depuis Python
- Modéliser des données pour MongoDB

---

🎉 **Félicitations !** Tu comprends maintenant le Big Data, les systèmes distribués et le NoSQL.